In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)

%matplotlib inline

In [73]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df[df["user_id"] % 100 == 0]
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [74]:
df["previous_tag"] = df.groupby("user_id")["tag"].shift(1)

In [75]:
df.columns

Index(['timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'user_answer', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'bundle_id', 'correct_answer', 'part', 'tags', 'tag', 'type_of',
       'previous_tag'],
      dtype='object')

In [76]:
df["part"].value_counts()

5    437040
2    192162
6    112269
3     91982
4     85210
1     80653
7     52232
Name: part, dtype: int64

In [77]:
df["bundle_id"].value_counts()

-1        20470
 7216      6228
 2063      5112
 3363      5025
 2946      4920
          ...  
 5767         1
 9726         1
 12723        1
 933          1
 12728        1
Name: bundle_id, Length: 9530, dtype: int64

In [78]:
for c in ["content_id", "task_container_id", "part", "bundle_id"]:
    print("\n\n\n==================================")
    print(c)
    print("==================================")
    df[f"previous_{c}"] = df.groupby("user_id")[c].shift(1)
    df[f"previous_2_{c}"] = df.groupby("user_id")[c].shift(2)
    df[f"equal_{c}"] = df[c] == df[f"previous_{c}"]
    df[f"equal_2_{c}"] = (df[c] == df[f"previous_{c}"]) * (df[c] == df[f"previous_2_{c}"])
    print(df.groupby(f"equal_{c}")["answered_correctly"].describe())
    print(df.groupby(f"equal_2_{c}")["answered_correctly"].describe())




content_id
                      count      mean       std  min  25%  50%  75%  max
equal_content_id                                                        
False             1028595.0  0.648794  0.477348  0.0  0.0  1.0  1.0  1.0
True                 2483.0  0.625050  0.484207  0.0  0.0  1.0  1.0  1.0


/home/owner/anaconda3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


                        count      mean       std  min  25%  50%  75%  max
equal_2_content_id                                                        
False               1031072.0  0.648735  0.477366  0.0  0.0  1.0  1.0  1.0
True                      6.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0



task_container_id
                            count      mean       std  min  25%  50%  75%  max
equal_task_container_id                                                       
False                    792732.0  0.646012  0.478206  0.0  0.0  1.0  1.0  1.0
True                     238346.0  0.657800  0.474447  0.0  0.0  1.0  1.0  1.0


/home/owner/anaconda3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


                              count      mean       std  min  25%  50%  75%  \
equal_2_task_container_id                                                     
False                      890650.0  0.647554  0.477732  0.0  0.0  1.0  1.0   
True                       140428.0  0.656237  0.474965  0.0  0.0  1.0  1.0   

                           max  
equal_2_task_container_id       
False                      1.0  
True                       1.0  



part


/home/owner/anaconda3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


               count      mean       std  min  25%  50%  75%  max
equal_part                                                       
False        75550.0  0.631727  0.482339  0.0  0.0  1.0  1.0  1.0
True        955528.0  0.650081  0.476944  0.0  0.0  1.0  1.0  1.0
                 count      mean       std  min  25%  50%  75%  max
equal_2_part                                                       
False         142880.0  0.629003  0.483073  0.0  0.0  1.0  1.0  1.0
True          888198.0  0.651911  0.476365  0.0  0.0  1.0  1.0  1.0



bundle_id
                    count      mean       std  min  25%  50%  75%  max
equal_bundle_id                                                       
False            790020.0  0.646049  0.478194  0.0  0.0  1.0  1.0  1.0
True             241058.0  0.657543  0.474533  0.0  0.0  1.0  1.0  1.0
                      count      mean       std  min  25%  50%  75%  max
equal_2_bundle_id                                                       
False              89

/home/owner/anaconda3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [79]:
df["previous_timestamp"] = df.groupby("user_id")["timestamp"].shift(1)

In [80]:
df["diff_timestamp"] = df["timestamp"] - df["previous_timestamp"]

In [81]:
df = df.reset_index(drop=True)

In [84]:
df["diff_bin_timestamp"] = pd.cut(df["diff_timestamp"], [-1, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10])

In [85]:
df.groupby("diff_bin_timestamp")["answered_correctly"].describe()

,count,mean,std,min,25%,50%,75%,max
diff_bin_timestamp,,,,,,,,
"(-1, 1000]",238591.0,0.657464,0.474559,0.0,0.0,1.0,1.0,1.0
"(1000, 10000]",8175.0,0.569297,0.495205,0.0,0.0,1.0,1.0,1.0
"(10000, 100000]",569544.0,0.659410,0.473908,0.0,0.0,1.0,1.0,1.0
"(100000, 1000000]",148429.0,0.612380,0.487209,0.0,0.0,1.0,1.0,1.0
"(1000000, 10000000]",20762.0,0.619401,0.485546,0.0,0.0,1.0,1.0,1.0
"(10000000, 100000000]",26961.0,0.618783,0.485695,0.0,0.0,1.0,1.0,1.0
"(100000000, 1000000000]",11903.0,0.600605,0.489795,0.0,0.0,1.0,1.0,1.0
"(1000000000, 10000000000]",2337.0,0.560976,0.496374,0.0,0.0,1.0,1.0,1.0


In [86]:
df["bin_timestamp"] = pd.cut(df["timestamp"], [-1, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10])

In [87]:
df.groupby("bin_timestamp")["answered_correctly"].describe()

,count,mean,std,min,25%,50%,75%,max
bin_timestamp,,,,,,,,
"(-1, 1000]",3977.0,0.672115,0.469502,0.0,0.0,1.0,1.0,1.0
"(1000, 10000]",67.0,0.477612,0.503268,0.0,0.0,0.0,1.0,1.0
"(10000, 100000]",12054.0,0.497760,0.500016,0.0,0.0,0.0,1.0,1.0
"(100000, 1000000]",57777.0,0.469824,0.499093,0.0,0.0,0.0,1.0,1.0
"(1000000, 10000000]",27917.0,0.615969,0.486374,0.0,0.0,1.0,1.0,1.0
"(10000000, 100000000]",43956.0,0.644008,0.478818,0.0,0.0,1.0,1.0,1.0
"(100000000, 1000000000]",196725.0,0.655362,0.475251,0.0,0.0,1.0,1.0,1.0
"(1000000000, 10000000000]",440137.0,0.666954,0.471303,0.0,0.0,1.0,1.0,1.0
